# Criação de Banco de Dados e Inserção de Dados no PostgreSQL

### 1. Criação do Banco de Dados
Crie um novo banco de dados chamado `ecommerce` utilizando o seguinte comando SQL:
```sql
CREATE DATABASE ecommerce;
```

### 2. Criação de outras tabelas:

Tabela produtos:
```sql
CREATE TABLE Produtos (
    ID_Produto SERIAL PRIMARY KEY,
    Nome TEXT NOT NULL,
    Descricao TEXT,
    Preco DECIMAL(10, 2) NOT NULL,
    Estoque INTEGER,
    Categoria VARCHAR(255)
);
```

Tabela de Clientes
```sql
CREATE TABLE Clientes (
    ID_Cliente SERIAL PRIMARY KEY,
    Nome TEXT NOT NULL,
    Endereco TEXT,
    Telefone VARCHAR(50),
    Email VARCHAR(255),
    Data_Nascimento DATE
);
```
Tabela de Pedidos
```sql
CREATE TABLE Pedidos (
    ID_Pedido SERIAL PRIMARY KEY,
    Data_Pedido DATE NOT NULL,
    Status VARCHAR(50),
    Valor_Total DECIMAL(10, 2) NOT NULL,
    ID_Cliente INTEGER,
    FOREIGN KEY (ID_Cliente) REFERENCES Clientes(ID_Cliente)
);
```

Tabela de Itens_Pedido
```sql
CREATE TABLE Itens_Pedido (
    ID_Item SERIAL PRIMARY KEY,
    Quantidade INTEGER NOT NULL,
    Preco_Unitario DECIMAL(10, 2) NOT NULL,
    Subtotal DECIMAL(10, 2) NOT NULL,
    ID_Pedido INTEGER,
    ID_Produto INTEGER,
    FOREIGN KEY (ID_Pedido) REFERENCES Pedidos(ID_Pedido),
    FOREIGN KEY (ID_Produto) REFERENCES Produtos(ID_Produto)
);
```

Tabela de Pagamentos:
```sql
CREATE TABLE Pagamentos (
    ID_Pagamento SERIAL PRIMARY KEY,
    Data_Pagamento DATE NOT NULL,
    Valor_Pago DECIMAL(10, 2) NOT NULL,
    Metodo_Pagamento VARCHAR(50),
    ID_Pedido INTEGER,
    FOREIGN KEY (ID_Pedido) REFERENCES Pedidos(ID_Pedido)
);
```

### 3. Inserção de Dados com Python:

### Script Python para Inserção de Dados:

In [ ]:
pip install psycopg2 Faker

In [1]:
import psycopg2
import random
from faker import Faker

In [9]:
# Configurações de conexão com o banco de dados
conn = psycopg2.connect(
    dbname="dbname",
    user="user",
    password="password",
    host="localhost",
    port="port"
)
cursor = conn.cursor()

In [7]:
fake = Faker()

In [10]:

# Função para truncar strings longas
def truncate_string(s, max_length):
    return s[:max_length] if len(s) > max_length else s

# Função para inserir dados na tabela de Clientes
def insert_clients(n):
    for _ in range(n):
        cursor.execute(
            "INSERT INTO Clientes (Nome, Endereco, Telefone, Email, Data_Nascimento) VALUES (%s, %s, %s, %s, %s)",
            (
                truncate_string(fake.name(), 255),
                truncate_string(fake.address(), 255),
                truncate_string(fake.phone_number(), 50),
                truncate_string(fake.email(), 255),
                fake.date_of_birth()
            )
        )
    conn.commit()

# Função para inserir dados na tabela de Produtos
def insert_products(n):
    categories = ['Eletrônicos', 'Roupas', 'Livros', 'Alimentos', 'Móveis']
    for _ in range(n):
        cursor.execute(
            "INSERT INTO Produtos (Nome, Descricao, Preco, Estoque, Categoria) VALUES (%s, %s, %s, %s, %s)",
            (
                truncate_string(fake.word(), 255),
                truncate_string(fake.sentence(), 255),
                round(random.uniform(10.0, 1000.0), 2),
                random.randint(1, 100),
                random.choice(categories)
            )
        )
    conn.commit()

# Função para obter todos os IDs da tabela Clientes
def get_client_ids():
    cursor.execute("SELECT ID_Cliente FROM Clientes")
    return [row[0] for row in cursor.fetchall()]

# Função para obter todos os IDs da tabela Produtos
def get_product_ids():
    cursor.execute("SELECT ID_Produto FROM Produtos")
    return [row[0] for row in cursor.fetchall()]

# Função para inserir dados na tabela de Pedidos
def insert_orders(n, client_ids):
    for _ in range(n):
        cursor.execute(
            "INSERT INTO Pedidos (Data_Pedido, Status, Valor_Total, ID_Cliente) VALUES (%s, %s, %s, %s)",
            (
                fake.date_this_year(),
                truncate_string(random.choice(['Pendente', 'Concluído', 'Cancelado']), 50),
                round(random.uniform(20.0, 5000.0), 2),
                random.choice(client_ids)
            )
        )
    conn.commit()

# Função para inserir dados na tabela de Itens_Pedido
def insert_order_items(n, order_ids, product_ids):
    for _ in range(n):
        id_pedido = random.choice(order_ids)
        id_produto = random.choice(product_ids)
        quantidade = random.randint(1, 10)
        preco_unitario = round(random.uniform(10.0, 1000.0), 2)
        subtotal = round(preco_unitario * quantidade, 2)
        cursor.execute(
            "INSERT INTO Itens_Pedido (Quantidade, Preco_Unitario, Subtotal, ID_Pedido, ID_Produto) VALUES (%s, %s, %s, %s, %s)",
            (
                quantidade,
                preco_unitario,
                subtotal,
                id_pedido,
                id_produto
            )
        )
    conn.commit()

# Função para obter todos os IDs da tabela Pedidos
def get_order_ids():
    cursor.execute("SELECT ID_Pedido FROM Pedidos")
    return [row[0] for row in cursor.fetchall()]

# Função para inserir dados na tabela de Pagamentos
def insert_payments(n, order_ids):
    for _ in range(n):
        cursor.execute(
            "INSERT INTO Pagamentos (Data_Pagamento, Valor_Pago, Metodo_Pagamento, ID_Pedido) VALUES (%s, %s, %s, %s)",
            (
                fake.date_this_year(),
                round(random.uniform(20.0, 5000.0), 2),
                truncate_string(random.choice(['Cartão de Crédito', 'Boleto', 'Transferência']), 50),
                random.choice(order_ids)
            )
        )
    conn.commit()

# Inserindo os dados
insert_clients(10000)
insert_products(10000)
client_ids = get_client_ids()
insert_orders(10000, client_ids)
order_ids = get_order_ids()
product_ids = get_product_ids()
insert_order_items(10000, order_ids, product_ids)
insert_payments(10000, order_ids)

In [11]:
# Fechando a conexão
cursor.close()
conn.close()